In [1]:
import random

import numpy as np
import torch
from torch.utils.data.dataloader import default_collate

from settings import EXPERIMENTS_DIR
from experiment import Experiment
from utils2 import to_device, load_weights, load_embeddings, create_embeddings_matrix
from vocab import Vocab
from train2 import create_model
from preprocess import load_dataset, create_dataset_reader

In [2]:
import argparse
import json
from models import MLPClassifier, Baseline_Embeddings
from models import Seq2Seq, MLP_D, MLP_G, MLP_I, MLP_I_AE, JSDistance, Seq2SeqCAE, Baseline_Embeddings, Baseline_LSTM
from utils import to_gpu, Corpus, batchify, SNLIDataset, collate_snli
import random
import pickle as pkl
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


In [3]:
exp_id = 'train.jaw89lze'

# Load everything

In [4]:
EXPERIMENTS_DIR

PosixPath('data/experiments')

In [5]:
exp = Experiment.load(EXPERIMENTS_DIR, exp_id)

In [6]:
preprocess_exp = Experiment.load(EXPERIMENTS_DIR, exp.config.preprocess_exp_id)
dataset_train, dataset_val, dataset_test, vocab, style_vocab, W_emb = load_dataset(preprocess_exp)

Dataset: 453655, val: 10000, test: 10000
Vocab: 9419, style vocab: 2
W_emb: (9419, 300)


In [7]:
dataset_reader = create_dataset_reader(preprocess_exp.config)

In [8]:
model = create_model(exp.config, vocab, style_vocab, dataset_train.max_len, W_emb)

In [9]:
dataset_train.max_len

20

In [10]:
load_weights(model, exp.experiment_dir.joinpath('best.th'))

In [11]:
model = model.eval()

## Predict

In [12]:
def create_inputs(instances):
    if not isinstance(instances, list):
        instances = [instances,]
        
    if not isinstance(instances[0], dict):
        sentences = [
            dataset_reader.preprocess_sentence(dataset_reader.spacy( dataset_reader.clean_sentence(sent)))
            for sent in instances
        ]
        
        style = list(style_vocab.token2id.keys())[0]
        instances = [
            {
                'sentence': sent,
                'style': style,
            }
            for sent in sentences
        ]
#         print("INST!")
        for inst in instances:
#             print(inst)
            inst_encoded = dataset_train.encode_instance(inst)
            inst.update(inst_encoded)            
    
    
    instances = [
        {
            'sentence': inst['sentence_enc'],
            'style': inst['style_enc'],
        } 
        for inst in instances
    ]
    
    instances = default_collate(instances)
    instances = to_device(instances)      
    
    return instances

In [13]:
def get_sentences(outputs):
    predicted_indices = outputs["predictions"]
    end_idx = vocab[Vocab.END_TOKEN]
    
    if not isinstance(predicted_indices, np.ndarray):
        predicted_indices = predicted_indices.detach().cpu().numpy()

    all_predicted_tokens = []
    for indices in predicted_indices:
        indices = list(indices)

        # Collect indices till the first end_symbol
        if end_idx in indices:
            indices = indices[:indices.index(end_idx)]

        predicted_tokens = [vocab.id2token[x] for x in indices]
        all_predicted_tokens.append(predicted_tokens)
        
    return all_predicted_tokens

In [14]:
sentence =  ' '.join(dataset_val.instances[1]['sentence'])

In [15]:
sentence

'they are really good people .'

In [16]:
inputs = create_inputs(sentence)

In [17]:
inputs

{'sentence': tensor([[136,  55,  29, 380, 368,   8,   2,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0]], device='cuda:0'),
 'style': tensor([0], device='cuda:0')}

In [18]:
outputs = model(inputs)

In [19]:
sentences = get_sentences(outputs)

In [20]:
' '.join(sentences[0])

'they are really good people .'

### Swap style

In [21]:
possible_styles = list(style_vocab.token2id.keys()) #['negative', 'positive']

In [22]:
style_vocab.token2id.keys()

dict_keys(['negative', 'positive'])

In [23]:
possible_styles

['negative', 'positive']

In [24]:
sentences0 = [s for s in dataset_val.instances if s['style'] == possible_styles[0]]
sentences1 = [s for s in dataset_val.instances if s['style'] == possible_styles[1]]

In [25]:
for i in np.random.choice(np.arange(len(sentences0)), 5):
    print(i, ' '.join(sentences0[i]['sentence']))

535 the lady at the counter was friendly and took our order w/ no problems .
3659 it was not very busy for saturday night .
3933 this evening 's dinner was a disaster !
3311 service was great but the number oz bone in ribeye was very disappointing .
2035 not even worth the one star rating we are forced to give !


In [26]:
for i in np.random.choice(np.arange(len(sentences1)), 5):
    print(i, ' '.join(sentences1[i]['sentence']))

3948 she was great .
1506 other than that everything else is great !
4390 the biscuits here are always the perfect balance between crunchy and thick .
700 i also tried the spinach and egg bagel , also very good !
545 our guests raved and raved about how beautiful and delicious the cake was !


#### Swap

In [27]:
print(3874, ' '.join(sentences0[3874]['sentence']))

3874 the rice had hard things in it .


In [28]:
target0 = 3874 # np.random.choice(np.arange(len(sentences0)))
target1 = 4935 # np.random.choice(np.arange(len(sentences0)))

In [29]:
print(' '.join(sentences0[target0]['sentence']))

the rice had hard things in it .


In [30]:
print(' '.join(sentences1[target1]['sentence']))

which is awesome !


In [31]:
inputs = create_inputs([
    sentences0[target0],
    sentences1[target1],
])

In [32]:
z_hidden = model(inputs)

In [33]:
z_hidden['style_hidden'].shape

torch.Size([2, 128])

In [34]:
z_hidden['meaning_hidden'].shape

torch.Size([2, 128])

In [35]:
original_decoded = model.decode(z_hidden)

In [36]:
original_sentences = get_sentences(original_decoded)

In [37]:
print(' '.join(original_sentences[0]))
print(' '.join(original_sentences[1]))

the rice had hard things in it .
which is awesome !


In [38]:
test = model(create_inputs('sophistication extraordinary asdsadasdadsadasdsadasdsadasdadsadsd <unk>'))

In [39]:
get_sentences(model.decode(test))

[['everything', ',', ',', 'with', 'the', 'experience', 'of', 'gorgeous', '.']]

In [40]:
z_hidden['meaning_hidden'][1]

tensor([ 0.0304,  0.3485, -0.1832,  0.0168,  0.2922, -0.2489,  0.4045,  1.0091,
         0.0626, -0.0538,  0.2468,  0.0552, -0.1199, -0.1482,  0.0519, -0.3881,
         0.0385,  0.0611, -0.2959, -0.0111,  0.4923,  0.1755,  0.2372, -0.1484,
        -0.2288, -0.1240, -0.2253, -0.0819, -0.2461, -0.0367,  0.2760,  0.1065,
         0.0356, -0.1010, -0.1400,  0.2693, -0.2108, -0.1767,  0.4459,  0.4920,
        -0.5315,  0.2920, -0.2592,  0.0526,  0.0453, -0.3169,  0.0054,  0.0895,
         0.2257, -0.0154, -0.1207, -0.1999,  0.0278,  0.1244, -0.0796,  0.0468,
         0.0466, -0.1769, -0.3507, -0.2137,  0.2887,  0.2937, -0.2503,  0.1329,
        -0.1793,  0.1662,  0.1337,  0.4106, -0.1068, -0.0255, -0.2377,  0.1682,
        -0.0984,  0.4288, -0.0108, -0.1960,  0.1689,  0.2532, -0.2159, -0.1833,
        -0.0642,  0.3634,  0.1607,  0.0532, -0.0969, -0.2231, -0.0929,  0.1783,
         0.2294, -0.1193, -0.4915, -0.2499, -0.1053, -0.3476,  0.6208, -0.0799,
        -0.0934,  0.1069,  0.0144, -0.16

In [41]:
for i in range (0, z_hidden['style_hidden'][0].shape[0]):
    z_hidden['style_hidden'][0][i] = 0.9


In [42]:
z_hidden['style_hidden'][0].shape[0]

128

In [43]:
z_hidden['style_hidden'][0]

tensor([0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 

In [44]:
z_hidden_swapped = {
    'meaning_hidden': torch.stack([
        z_hidden['meaning_hidden'][0].clone(),
        #z_hidden['meaning_hidden'][1].clone(),        
    ], dim=0),
    'style_hidden': torch.stack([
        z_hidden['style_hidden'][0].clone(),
        #z_hidden['style_hidden'][0].clone(),        
    ], dim=0),
}

In [45]:
swaped_decoded = model.decode(z_hidden_swapped)

In [46]:
swaped_sentences = get_sentences(swaped_decoded)

In [47]:
print(' '.join(original_sentences[0]))
# print(' '.join(original_sentences[1]))
print()
print(' '.join(swaped_sentences[0]))
# print(' '.join(swaped_sentences[1]))

the rice had hard things in it .

i am only going for that like anyone !


In [48]:
#python3.6 train_surrogate.py --data_path ./data/classifier --save_path game_output/ --classifier_path ./data --load_pretrained .
cur_dir = '.'

with open(cur_dir + '/vocab.json', 'r') as fin:
    corpus_vocab = json.load(fin)

corpus_train = SNLIDataset(train=True, vocab_size=11004-4, path='./data/classifier')
corpus_test = SNLIDataset(train=False, vocab_size=11004-4, path='./data/classifier')
trainloader= torch.utils.data.DataLoader(corpus_train, batch_size = 32, collate_fn=collate_snli, shuffle=True)
train_iter = iter(trainloader)
testloader= torch.utils.data.DataLoader(corpus_test, batch_size = 32, collate_fn=collate_snli, shuffle=False)
random.seed(1111)
np.random.seed(1111)
torch.manual_seed(1111)

EPS = 3e-2


original vocab 41574; pruned to 11004
Number of sentences dropped from ./data/classifier/train.txt: 448221 out of 549367 total
original vocab 41574; pruned to 11004
Number of sentences dropped from ./data/classifier/test.txt: 8288 out of 9824 total


In [49]:
autoencoder = torch.load(open(cur_dir + '/models/autoencoder_model.pt', 'rb'))
#gan_gen = torch.load(open(cur_dir + '/models/gan_gen_model.pt', 'rb'))
#gan_disc = torch.load(open(cur_dir + '/models/gan_disc_model.pt', 'rb'))
inverter = torch.load(open(cur_dir + '/models/inverter_model.pt', 'rb'))

classifier1 = Baseline_Embeddings(100, vocab_size=11004)
#classifier1 = Baseline_LSTM(100,300,maxlen=args.maxlen, gpu=args.cuda)
classifier1.load_state_dict(torch.load('./models' + "/baseline/model_emb.pt"))
vocab_classifier1 = pkl.load(open('./models' + "/vocab.pkl", 'rb'))

mlp_classifier = MLPClassifier(100 * 2, 3, layers='100-50')
#if not args.train_mode:
mlp_classifier.load_state_dict(torch.load('./game_output'+'/surrogate{0}.pt'.format('100-50')))

print(classifier1)
print(autoencoder)
print(inverter)
print(mlp_classifier)

optimizer = optim.Adam(mlp_classifier.parameters(),
                           lr=1e03,
                           betas=(0.9, 0.999))


/home/game/.local/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'models.Seq2SeqCAE' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Baseline_Embeddings(
  (embedding_prem): Embedding(11004, 100)
  (embedding_hypo): Embedding(11004, 100)
  (linear): Linear(in_features=200, out_features=3, bias=True)
)
Seq2SeqCAE(
  (embedding): Embedding(11004, 300)
  (embedding_decoder): Embedding(11004, 300)
  (encoder): Sequential(
    (layer-1): Conv1d(300, 500, kernel_size=(3,), stride=(1,))
    (bn-1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation-1): LeakyReLU(negative_slope=0.2, inplace)
    (layer-2): Conv1d(500, 700, kernel_size=(3,), stride=(2,))
    (bn-2): BatchNorm1d(700, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation-2): LeakyReLU(negative_slope=0.2, inplace)
    (layer-3): Conv1d(700, 1000, kernel_size=(3,), stride=(2,))
    (bn-3): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation-3): LeakyReLU(negative_slope=0.2, inplace)
  )
  (linear): Linear(in_features=1000, out_features=300, bias=

In [50]:
from torch.autograd import Variable

def evaluate_model():
    classifier1.eval()

    test_iter = iter(trainloader)
    correct=0
    total=0
    for batch in test_iter:
        premise, hypothesis, target, _, _, _, _ = batch

        if args.cuda:
            premise=premise.cuda()
            hypothesis = hypothesis.cuda()
            target = target.cuda()

        prob_distrib = classifier1.forward((premise, hypothesis))
        predictions = np.argmax(prob_distrib.data.cpu().numpy(), 1)
        correct+=len(np.where(target.data.cpu().numpy()==predictions)[0])
        total+=premise.size(0)
    acc=correct/float(total)
    print("Accuracy:{0}".format(acc))
    return acc

autoencoder.gpu = True
autoencoder = autoencoder.cuda()
autoencoder.start_symbols = autoencoder.start_symbols.cuda()
#gan_gen = gan_gen.cuda()
#gan_disc = gan_disc.cuda()
classifier1 = classifier1.cuda()
inverter = inverter.cuda()
mlp_classifier = mlp_classifier.cuda()


In [51]:
def train_process(premise, hypothesis, target, premise_words, hypothesis_words, premise_length, hypothesis_length):
    #mx = target.max().item()
    #assert(mx >= 0 and mx < 3)
    #for s, s_w in zip(premise, premise_words):
    #    for i, w in zip(s, s_w):
    #        assert(corpus_vocab.get(w, 3) == i)
    #print(hypothesis_words, flush=True)
    autoencoder.eval()
    inverter.eval()
    classifier1.eval()
    mlp_classifier.train()

    #print(premise.max().item(), flush=True)
    #print(hypothesis.max().item(), flush=True)

    premise_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in premise_words]).cuda()
    hypothesis_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in hypothesis_words]).cuda()

    c_prem = autoencoder.encode(premise_idx, premise_length, noise=False)
    z_prem = inverter(c_prem).detach()

    c_hypo = autoencoder.encode(hypothesis_idx, hypothesis_length, noise=False)
    z_hypo = inverter(c_hypo).detach()

    # z_comb = nn.cat((z_prem, z_hypo), 0).detach()

    output = mlp_classifier(z_prem, z_hypo)
    gold = classifier1((premise, hypothesis)).detach()

    #print(output.shape, flush=True)
    #print(gold.shape, flush=True)

    acc = (torch.argmax(gold, 1) == target).to(torch.float32).mean().item()
    acc_surrogate = (torch.argmax(output, 1) == target).to(torch.float32).mean().item()


    loss = -torch.mean(torch.sum(output * F.softmax(gold, dim=1), 1), 0)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item(), acc, acc_surrogate


In [58]:
def perturb(criterion, premise, hypothesis, target, premise_words, hypothesis_words, premise_length, hypothesis_length):
    autoencoder.eval()
    inverter.eval()
    classifier1.eval()
    mlp_classifier.eval()

    premise_words = [premise_words]
    hypothesis_words = [hypothesis_words]
    premise_length = [premise_length]
    hypothesis_length = [hypothesis_length]


    premise_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in premise_words]).cuda()
    hypothesis_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in hypothesis_words]).cuda()
#     print('premise_idx = ' + str(premise_idx))
#     print('hypothesis-idx = ' + str(hypothesis_idx))
#     c_prem = autoencoder.encode(premise_idx, premise_length, noise=False)
    c_prem = create_inputs(premise_words[0])
    print("C_PREM")
    print(c_prem)
#     c_prem = create_inputs(' '.join(premise_words[0]))
    z_prem = inverter(c_prem).detach()
#     print('c_prem = ' + str(c_prem))
#     print('z_prem = ' + str(z_prem))
    c_hypo = autoencoder.encode(hypothesis_idx, hypothesis_length, noise=False).detach()
    c_hypo.requires_grad = True
    z_hypo = inverter(c_hypo)


    premise = premise.unsqueeze(0)
    hypothesis = hypothesis.unsqueeze(0)
    target = target.unsqueeze(0)

    output = mlp_classifier(z_prem, z_hypo)

    loss = criterion(output, target)
    mlp_classifier.zero_grad()
    inverter.zero_grad()
    loss.backward()

    direction = torch.sign(c_hypo.grad)
    nc_hypo = c_hypo + EPS * direction
    nhypo_idx = autoencoder.generate(nc_hypo, 10, False)

    return nhypo_idx.squeeze(0).cpu().numpy()


In [53]:
def classifier_pred(pw, hw):
    classifier1.eval()

    premise_idx = torch.tensor([vocab_classifier1.get(w, 3) for w in pw]).cuda().unsqueeze(0)
    hypothesis_idx = torch.tensor([vocab_classifier1.get(w, 3) for w in hw]).cuda().unsqueeze(0)

    return F.softmax(classifier1((premise_idx, hypothesis_idx)), 1).squeeze(0).cpu().detach().numpy()


In [54]:
def maximum(arr):
    a = -1
    t = 0
    for i in range(len(arr)):
        if arr[i] > a:
            a = arr[i]
            t = i
    return t


In [55]:
Vocab.UNK_TOKEN

'<unk>'

In [59]:
criterion = nn.CrossEntropyLoss().cuda()

niter = 0
evaluate = 0
nevaluate = 0
idx2words = dict(map(lambda x: (x[1], x[0]), corpus_vocab.items()))
while niter < len(testloader):
    niter += 1
    batch = train_iter.next()
    for p, h, t, pw, hw, pl, hl in zip(*batch):
        nh = perturb(criterion, p.cuda(), h.cuda(), t.cuda(), pw, hw, pl, hl)
        print('--------------------------------')
        print('Target ', t)
#         print(' '.join(pw))
#         temp = ' '.join(pw)
        nhw = (['<sos>'] + [idx2words[i] for i in nh])[:10]
        print(' '.join(nhw))
        print('Old ', classifier_pred(pw, hw))
        print('New ', classifier_pred(pw, nhw))
        print('Old Pred: ', maximum(classifier_pred(pw, hw)))
        print('New Pred: ', maximum(classifier_pred(pw, nhw)))
        print('Good: ', maximum(classifier_pred(pw, hw)) != maximum(classifier_pred(pw, nhw)))
        evaluate = evaluate + int(maximum(classifier_pred(pw, hw)) != maximum(classifier_pred(pw, nhw)))
        nevaluate = nevaluate + 1
    break
print(evaluate / nevaluate)
print(nevaluate)



C_PREM
{'sentence': tensor([[   0,    0,    0,    2,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 747,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [3652,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 726,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [  24,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [1525,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
       

AttributeError: 'dict' object has no attribute 'dim'

In [57]:
np.array(vocab.get('aasdasdasdsad', 0), dtype=np.long)

array(0)